In [1]:
import pandas as pd
import requests
from collections import Counter
from sf_parks.models import FilmLocations, PrivateOpenSpaces
from sf_parks.data import (
    insert_into_db,
    film_locations,
    park_scores,
    private_open_spaces,
    properties
)

pd.set_option('display.max_rows', 100)

## Check contents of datasets

In [2]:
def check_key_counts(json_data):
    print(f'number of records: {len(json_data)}')
    keys = Counter()
    for record in json_data:
        for k in record:
            keys[k] += 1
    return keys

In [3]:
check_key_counts(film_locations)

number of records: 1000


Counter({'title': 1000,
         'release_year': 1000,
         'locations': 975,
         'production_company': 1000,
         'director': 1000,
         'writer': 1000,
         'actor_1': 996,
         'actor_2': 974,
         'actor_3': 790,
         'address': 975,
         'distributor': 945,
         'fun_facts': 236})

In [4]:
check_key_counts(park_scores)

number of records: 1000


Counter({'parkid': 1000, 'psa': 1000, 'park': 1000, 'fq': 1000, 'score': 1000})

In [5]:
check_key_counts(private_open_spaces)

number of records: 78


Counter({'name': 78,
         'popos_address': 78,
         'hours': 72,
         'type': 78,
         'landscaping': 70,
         'seating_no': 71,
         'food_service': 73,
         'art': 71,
         'restrooms': 72,
         'accessibility': 72,
         'location': 53,
         'year': 78,
         'description': 72,
         'seating_and_tables': 22,
         'hours_type': 76,
         'subject_to_downtown_pln': 78,
         'signage': 74,
         'block_num': 78,
         'lot_num': 78,
         'parcel_num': 78,
         'the_geom': 78,
         ':@computed_region_6qbp_sg9q': 78,
         ':@computed_region_qgnn_b9vv': 78,
         ':@computed_region_26cr_cadq': 78,
         ':@computed_region_ajp5_b2md': 78,
         ':@computed_region_rxqg_mtj9': 78,
         ':@computed_region_yftq_j783': 78,
         ':@computed_region_jx4q_fizf': 78,
         ':@computed_region_bh8s_q3mv': 78,
         ':@computed_region_jwn9_ihcz': 78,
         'latitude': 78,
         'longitude': 7

In [6]:
check_key_counts(properties)

number of records: 240


Counter({'objectid': 240,
         'map_label': 240,
         'longitude': 240,
         'latitude': 240,
         'acres': 240,
         'tma_propertyid': 240,
         'globalid': 240,
         'created_user': 240,
         'created_date': 240,
         'last_edited_user': 240,
         'last_edited_date': 240,
         'squarefeet': 240,
         'perimeterlength': 240,
         'propertytype': 240,
         'address': 240,
         'city': 240,
         'state': 240,
         'zipcode': 240,
         'complex': 240,
         'psa': 240,
         'supdist': 240,
         'ownership': 239,
         'land_id': 238,
         'ggp_section': 237,
         'state_senate': 237,
         'mons_neighborhood': 240,
         'police_district': 237,
         'us_congress': 237,
         'realtor_neighborhood': 239,
         'state_assembly': 237,
         'planning_neighborhood': 240,
         'shape': 240})

## Explore Datasets

In [7]:
film_locations_df = pd.DataFrame(film_locations)
park_scores_df = pd.DataFrame(park_scores)
private_open_spaces_df = pd.DataFrame(private_open_spaces)
properties_df = pd.DataFrame(properties)


`park_scores` and `properties` might be able to be joined with `park_scores.parkid` = `properties.tma_propertyid`. 


In [8]:
park_scores_and_prop_df = park_scores_df.set_index('parkid').join(properties_df.set_index('tma_propertyid'), rsuffix='_prop')
park_scores_and_prop_df

,psa,park,fq,score,objectid,map_label,longitude,latitude,acres,globalid,...,land_id,ggp_section,state_senate,mons_neighborhood,police_district,us_congress,realtor_neighborhood,state_assembly,planning_neighborhood,shape
1,PSA5,Glen Park,FY05Q4,0.85799999999999998,17064,Glen Canyon Park,-122.44266629,37.74020594,77.93628414,{9DBAB79F-33D3-48C4-B95B-756B4EE39079},...,8,N/A,11th Senatorial District,Glen Park,Ingleside,12th Congressional District,"Diamond Heights,Glen Park",17th Assembly District,Diamond Heights,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
1,PSA5,Glen Park,FY06Q2,0.99,17064,Glen Canyon Park,-122.44266629,37.74020594,77.93628414,{9DBAB79F-33D3-48C4-B95B-756B4EE39079},...,8,N/A,11th Senatorial District,Glen Park,Ingleside,12th Congressional District,"Diamond Heights,Glen Park",17th Assembly District,Diamond Heights,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
1,PSA5,Glen Park,FY06Q4,0.96299999999999997,17064,Glen Canyon Park,-122.44266629,37.74020594,77.93628414,{9DBAB79F-33D3-48C4-B95B-756B4EE39079},...,8,N/A,11th Senatorial District,Glen Park,Ingleside,12th Congressional District,"Diamond Heights,Glen Park",17th Assembly District,Diamond Heights,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
1,PSA5,Glen Park,FY07Q2,0.876,17064,Glen Canyon Park,-122.44266629,37.74020594,77.93628414,{9DBAB79F-33D3-48C4-B95B-756B4EE39079},...,8,N/A,11th Senatorial District,Glen Park,Ingleside,12th Congressional District,"Diamond Heights,Glen Park",17th Assembly District,Diamond Heights,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
1,PSA5,Glen Park,FY08Q2,1,17064,Glen Canyon Park,-122.44266629,37.74020594,77.93628414,{9DBAB79F-33D3-48C4-B95B-756B4EE39079},...,8,N/A,11th Senatorial District,Glen Park,Ingleside,12th Congressional District,"Diamond Heights,Glen Park",17th Assembly District,Diamond Heights,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,PSA1,Rochambeau Playground,FY06Q3,0.95799999999999996,33,Rochambeau Playground,-122.4845337,37.78483899,0.9086263,{2321DDCD-92E8-424B-8EDD-D82091A67FAD},...,190,N/A,11th Senatorial District,Lake Street,Richmond,12th Congressional District,Lake Street,19th Assembly District,Outer Richmond,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
99,PSA1,Rochambeau Playground,FY07Q1,0.754,33,Rochambeau Playground,-122.4845337,37.78483899,0.9086263,{2321DDCD-92E8-424B-8EDD-D82091A67FAD},...,190,N/A,11th Senatorial District,Lake Street,Richmond,12th Congressional District,Lake Street,19th Assembly District,Outer Richmond,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
99,PSA1,Rochambeau Playground,FY07Q3,0.92200000000000004,33,Rochambeau Playground,-122.4845337,37.78483899,0.9086263,{2321DDCD-92E8-424B-8EDD-D82091A67FAD},...,190,N/A,11th Senatorial District,Lake Street,Richmond,12th Congressional District,Lake Street,19th Assembly District,Outer Richmond,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
99,PSA1,Rochambeau Playground,FY08Q1,0.92600000000000005,33,Rochambeau Playground,-122.4845337,37.78483899,0.9086263,{2321DDCD-92E8-424B-8EDD-D82091A67FAD},...,190,N/A,11th Senatorial District,Lake Street,Richmond,12th Congressional District,Lake Street,19th Assembly District,Outer Richmond,"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."


In [9]:
private_open_spaces_df

,name,popos_address,hours,type,landscaping,seating_no,food_service,art,restrooms,accessibility,...,sf_find_neighborhoods,current_police_districts,current_supervisor_districts,analysis_neighborhoods,food,seating,restroom,amenities,indoor,special_amenities
0,1 Bush Street,1 Bush Street,Open at all times,Urban Garden,Willow Trees and planting. Interseting archite...,None besides steps.,No,Yes,No,Indirect,...,108,6,3,8,NaN,NaN,NaN,NaN,NaN,NaN
1,Crocker Galleria,165 Sutter St,"6AM-7PM, M-SAT",Sun Terrace,"8 large planter with trees, shrubbery around e...","24 benches measuring 7'8""",In building below.,No,Yes,Very Indirect,...,19,6,3,8,Y,Y,True,NaN,NaN,NaN
2,301 Howard St,301 Howard St,Open at all times,Urban Garden,NaN,NaN,NaN,NaN,NaN,NaN,...,32,1,10,8,NaN,NaN,NaN,NaN,NaN,NaN
3,71 Stevenson St,71 Stevenson St,Open at all times,Pedestrian Walkways,"Marble paving, potted plants, fountain",Evidence that there was seating during the day...,Multiple restaurants,No,No,Direct,...,108,6,10,8,Y,Y,NaN,NaN,NaN,NaN
4,100 California St,100 California St,Open at all times,Plaza,Pots of flowers,"16 movable chairs, 4 tables, 52' of bench seating",Yes,No,No,Direct,...,108,6,3,8,Y,Y,NaN,NaN,NaN,NaN
5,525 Market St,525 Market St,Open at all times,Urban Garden,"Plant wall, water feature, abundant street tre...","168' platform/box seating, 164' terraced steps...",Yes,Yes,No,Direct,...,108,6,10,8,Y,Y,NaN,NaN,NaN,NaN
6,100 Pine St,100 Pine St,NaN,Urban Garden,Planting and flowers,NaN,Yes.,NaN,NaN,NaN,...,108,6,3,8,Y,Y,NaN,NaN,NaN,NaN
7,180 Howard St,180 Howard St,"8AM - 6PM, M-F",Pedestrian Walkways,Trees and wall of planters with tall grass.,12 linear seats,Yes,No,No,Direct,...,108,1,10,8,Y,NaN,True,NaN,NaN,NaN
8,456 Montomery St,456 Montgomery St,Open at all times,Plaza,"Few spare plantings, water feature","72' linear seating, 8 movable chairs and 2 tables",No,Yes,No,Indirect,...,108,6,3,8,NaN,Y,NaN,NaN,NaN,NaN
9,49 Stevenson St,49 Stevenson St,Open at all times,Pedestrian Walkway,"No plants, some interesting paving","20 chairs, taken inside before 5pm at time of ...",Yes,Yes,Restaurant,Direct,...,108,6,10,8,Y,Y,NaN,NaN,NaN,NaN


In [10]:
park_scores_df

,parkid,psa,park,fq,score
0,86,PSA4,Carl Larsen Park,FY05Q3,0.79500000000000004
1,13,PSA4,Junipero Serra Playground,FY05Q3,0.95699999999999996
2,9,PSA4,Rolph Nicol Playground,FY05Q3,0.86399999999999999
3,117,PSA2,Alamo Square,FY05Q4,0.85699999999999998
4,60,PSA6,Jose Coronado Playground,FY05Q4,0.85899999999999999
...,...,...,...,...,...
995,148,PSA1,Michelangelo Playground,FY08Q2,0.96299999999999997
996,109,PSA1,Presidio Heights Playground,FY08Q2,0.90700000000000003
997,170,PSA1,Woh Hei Yuen Park,FY08Q2,0.92300000000000004
998,182,PSA2,Fillmore-Turk Mini Park,FY08Q2,0.63600000000000001


In [11]:
private_open_spaces_df.set_index(':@computed_region_yftq_j783').join(park_scores_df.set_index('parkid')).to_csv('temp.csv')

In [12]:
ps =private_open_spaces[0]
ps

{'name': '1 Bush Street',
 'popos_address': '1 Bush Street',
 'hours': 'Open at all times',
 'type': 'Urban Garden',
 'landscaping': 'Willow Trees and planting. Interseting architectural design',
 'seating_no': 'None besides steps.',
 'food_service': 'No',
 'art': 'Yes',
 'restrooms': 'No',
 'accessibility': 'Indirect',
 'location': 'Below street level',
 'year': '1959',
 'description': 'This Urban Garden provides a sunken space featuring a water fountain. Space is accessed with stairs from Bush Street and curved walkways from Market street.',
 'seating_and_tables': 'N',
 'hours_type': 'Open At All Times',
 'subject_to_downtown_pln': 'No',
 'signage': 'None',
 'block_num': '0290',
 'lot_num': '011',
 'parcel_num': '0290011',
 'the_geom': {'type': 'Point', 'coordinates': [-122.40045257, 37.79098054]},
 ':@computed_region_6qbp_sg9q': '108',
 ':@computed_region_qgnn_b9vv': '6',
 ':@computed_region_26cr_cadq': '3',
 ':@computed_region_ajp5_b2md': '8',
 ':@computed_region_rxqg_mtj9': '10',


In [5]:
PrivateOpenSpaces(**ps)

PrivateOpenSpaces(id=None, name='1 Bush Street', popos_address='1 Bush Street', hours='Open at all times', type='Urban Garden', landscaping='Willow Trees and planting. Interseting architectural design', seating_no='None besides steps.', food_service='No', art='Yes', restrooms='No', accessibility='Indirect', location='Below street level', year=1959, description='This Urban Garden provides a sunken space featuring a water fountain. Space is accessed with stairs from Bush Street and curved walkways from Market street.', hours_type='Open At All Times', subject_to_downtown_pln='No', signage='None', block_num=290, lot_num=11, parcel_num=290011, latitude='-122.40045257', longitude='37.79098054', sf_find_neighborhoods=108, current_police_districts=6, current_supervisor_districts=3, analysis_neighborhoods=8)

In [ ]:
from get_data import insert_into_db

In [ ]:
fl = film_locations[0:2]
fl

In [ ]:
insert_into_db(fl, FilmLocations)

In [ ]:
film_locations_df[film_locations_df['title'].isna()]

In [ ]:
properties_df

In [ ]:
# film_locations_df['location_cleaned'] = film_locations_df['locations'].str.split("(")[0]
film_locations_df[['location_1', 'location_2']] = film_locations_df.locations.str.split("(", expand = True)
film_locations_df['location_2'].replace('\)','', inplace=True, regex=True)

In [ ]:
film_locations_df

In [ ]:
film_locations['locations'] 

In [ ]:
properties[0]

In [ ]:
private_open_spaces[0]